In [1]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import urllib
import urllib2
import pandas as pd
import numpy as np

In [13]:
r = urllib.urlopen('http://www.buda.org/leagues/past-leagues')
soup = BeautifulSoup(r, 'html.parser')

In [3]:
iFrames=[] # qucik bs4 example
for iframe in soup("iframe"):
    iFrames.append(soup.iframe.extract())

In [4]:
iFrames=[] # qucik bs4 example
iframexx = soup.find_all('iframe')
for iframe in iframexx:
    print iframe.find_all('html')

In [11]:
for iframe in iframexx:
    response = urllib2.urlopen(iframe.attrs['src'])
    iframe_soup = BeautifulSoup(response)

In [15]:
iframe = soup.find_all('iframe')[0]
response = urllib2.urlopen(iframe.attrs['src'])
iframe_soup = BeautifulSoup(response)

In [16]:
leaguelinks = [i.a['href'] for i in iframe_soup.find_all("td", class_="infobody")]

In [31]:
link = leaguelinks[15]

In [32]:
link.index('=')

29

In [154]:
leagueid = '38480'#link[link.index('league=') + 7:]

In [155]:
leagueid

'38480'

In [156]:
leaguescoreurl = 'http://www.buda.org/hatleagues/scores.php?section=showLeagueSchedule&league=' + leagueid + '&byDivision=1&showGames=0'

In [157]:
response = urllib2.urlopen(leaguescoreurl)
leaguescore_soup = BeautifulSoup(response)

In [89]:
data = []
table = leaguescore_soup.find_all('table', attrs={'class':'info'})[1]
# table_body = table.find('tbody')

rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('th')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [136]:
dfdata = pd.DataFrame(data)

In [137]:
dfdata.columns = ['Team', 'Record', 'Plus/Minus', 'Tourney Qualifying games']

In [138]:
dfdata = dfdata.drop(0).reset_index()

In [139]:
divnames = ['4/3 Div 1', '4/3 Div 2', '4/3 Div 3', 
            '5/2 Div 1', '5/2 Div 2', '5/2 Div 3', 
            'Open Div 1', 'Open Div 2']
divbases = [1800, 1400, 1000, 1600, 1200, 900, 1400, 1200]
dfdata['div'] = np.zeros(len(dfdata))
for i in range(len(divnames)-1):
    divstart = np.where(dfdata['Team'] == divnames[i])[0][0]
    divend = np.where(dfdata['Team'] == divnames[i + 1])[0][0]
#     print(divstart, divend)
    dfdata.ix[divstart + 1: divend, 'div'] = divbases[i]
dfdata.ix[divend + 1:, 'div'] = divbases[-1]


(0, 10)
(10, 29)
(29, 39)
(39, 45)
(45, 61)
(61, 77)
(77, 88)


In [140]:
for i in range(len(divnames)):
#     divstart = np.where(dfdata['Team'] == divnames[i])[0][0]
    dfdata = dfdata.drop(dfdata.index[dfdata['Team'] == divnames[i]])


In [141]:
# assert that an average goal differential per game of +5 gives +200 rating points.

In [142]:
dfdata['wins'] = dfdata['Record'].apply(lambda x: int(x.split('-')[0]))
dfdata['losses'] = dfdata['Record'].apply(lambda x: int(x.split('-')[1]))
dfdata['games'] = dfdata['wins'] + dfdata['losses']
dfdata['avgplusminus'] = dfdata['Plus/Minus'].astype('float') / dfdata['games']

In [143]:
dfdata['rating'] = dfdata['div'] + 60. * dfdata['avgplusminus']

In [172]:
teamsurl = 'http://www.buda.org/hatleagues/rosters.php?section=showTeams&league=' + leagueid

In [173]:
response = urllib2.urlopen(teamsurl)
teams_soup = BeautifulSoup(response)

In [184]:
teamlinks = [i.a['href'] for i in teams_soup.find_all("td", class_="infobody")]

TypeError: 'NoneType' object has no attribute '__getitem__'

In [180]:
tdlist = teams_soup.find_all('td', class_='infobody')

In [193]:
teamids = []
for i in tdlist:
    try:
        url = i.a['href']
        idindex = url.index('team=')
        whichindex = url.index('which=')
        teamids.append(url[idindex+5:whichindex-1])
    except:
        continue
# tdlist[1].a['href']

In [196]:
teamid = teamids[0]

In [161]:
iframe = teams_soup.find_all('iframe')[0]
response = urllib2.urlopen(iframe.attrs['src'])
iframe_teams_soup = BeautifulSoup(response)

IndexError: list index out of range

In [201]:
teamsurl = 'http://www.buda.org/hatleagues/rosters.php?section=showTeamRoster&team=' + teamid

In [202]:
teamsurl

'http://www.buda.org/hatleagues/rosters.php?section=showTeamRoster&team=38539'

In [203]:
response = urllib2.urlopen(teamsurl)
roster_soup = BeautifulSoup(response)

In [205]:
players = [i.get_text() for i in roster_soup.find_all("td", class_="infobody")]

In [206]:
players

[u'Beatty, Jen',
 u'Beaubien, Simone',
 u'Bernstein, Jeff',
 u'Bernstein, Tim',
 u'Blouwolff, Joshua',
 u'Brazile, Cory',
 u'Brazile, Rob',
 u'Brazile, Robert',
 u'Cooper, Amy',
 u'Demick, Kristen',
 u'Frankenberry, Ed',
 u'Goff, Marshall',
 u'Guarin, Justinne',
 u'Hussain, Talib',
 u'Kafka, Julie',
 u'levi, jonas',
 u'Peterson, Erik',
 u'Seton, Lewis',
 u'Stewart, Clara',
 u'']